# <a id='toc1_'></a>[Basket option implementation with Bachelier model CV](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Basket option implementation with Bachelier model CV](#toc1_)    
- [[To Do] Basket option implementation based on BSM model](#toc2_)    
  - [Write the similar test for BSM](#toc2_1_)    
- [Spread option implementation based on normal model](#toc3_)    
- [Spread option implementation based on BSM model](#toc4_)    
- [[To Do] Complete the implementation of basket_price_norm_analytic](#toc5_)    
- [Compare the MC stdev of BSM basket prices from with and without CV](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [2]:
%load_ext autoreload
%autoreload 2
'''
这两句的意思是当你导入了自己写的python模块时，在ipynb文件里import一次之后，即使你修改了你的python模块，重新运行import其实还是用的之前的python模块（因为已经将该模块在第一次导入时存在了内存里，第二次import没效果），这时需要重启kenel才能导入修改过的python模块。
上面这两条语句是自动加载你导入的python模块，保证你每次在ipynb里使用的模块是最新版本，所以在你修改了模块代码之后，不需要重新刷新你的kenel。
'''


'\n这两句的意思是当你导入了自己写的python模块时，在ipynb文件里import一次之后，即使你修改了你的python模块，重新运行import其实还是用的之前的python模块（因为已经将该模块在第一次导入时存在了内存里，第二次import没效果），这时需要重启kenel才能导入修改过的python模块。\n上面这两条语句是自动加载你导入的python模块，保证你每次在ipynb里使用的模块是最新版本，所以在你修改了模块代码之后，不需要重新刷新你的kenel。\n'

In [3]:
import numpy as np
import pyfeng as pf

#import option_models as opt
import basket

In [60]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100 
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5 # 资产的相关系数矩阵
texp = 5
strike = 120

In [61]:
cor_m

array([[1. , 0.5, 0.5, 0.5],
       [0.5, 1. , 0.5, 0.5],
       [0.5, 0.5, 1. , 0.5],
       [0.5, 0.5, 0.5, 1. ]])

In [62]:
print(weights)

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket)

[1 0 0 0]
26.570984651213593


In [63]:
# Compare the price to normal model formula

norm1 = pf.Norm(sigma=40)
price_norm = norm1.price(strike, spot[0], texp, cp=1)
print(price_basket, price_norm)

26.570984651213593 26.570845957870507


In [82]:
# A trivial test case 2
# all assets almost perfectly correlated:
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.0001*np.identity(4) + 0.9999*np.ones((4,4))
texp = 5
strike = 120

print( cor_m )

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_norm)

[[1.     0.9999 0.9999 0.9999]
 [0.9999 1.     0.9999 0.9999]
 [0.9999 0.9999 1.     0.9999]
 [0.9999 0.9999 0.9999 1.    ]]
26.57211110181949 26.570845957870507


In [88]:
# A full test set for basket option with exact price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 100
price_exact = 28.0073695

In [89]:
weights, cor_m

(array([0.25, 0.25, 0.25, 0.25]),
 array([[1. , 0.5, 0.5, 0.5],
        [0.5, 1. , 0.5, 0.5],
        [0.5, 0.5, 1. , 0.5],
        [0.5, 0.5, 0.5, 1. ]]))

In [87]:
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_exact)

28.095115748484492 28.0073695


# <a id='toc2_'></a>[[To Do] Basket option implementation based on BSM model](#toc0_)
## <a id='toc2_1_'></a>[Write the similar test for BSM](#toc0_)

In [68]:
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)
print(price_basket)

27.967309091912323


In [69]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)

[1 0 0 0]


In [49]:
bsm1 = pf.Bsm(sigma=vol[0])
price_bsm = bsm1.price(strike, spot[0], texp, cp=1)
print(price_basket, price_bsm)

28.51439365199859 28.713486748445934


# <a id='toc3_'></a>[Spread option implementation based on normal model](#toc0_)

In [70]:
# A full test set for spread option

spot = np.array([100, 96])
vol = np.array([0.2, 0.1])
weights = np.array([1, -1])
divr = np.array([1, 1])*0.05
intr = 0.1
cor = 0.5
cor_m = np.array([[1, cor], [cor, 1]])
texp = 1
strike = 0
price_exact = 8.5132252

In [71]:
# MC price based on normal model
# make sure that the prices are similar

np.random.seed(123456)
price_spread = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, bsm=False)
print(price_spread, price_exact)

8.317680907159142 8.5132252


# <a id='toc4_'></a>[Spread option implementation based on BSM model](#toc0_)

In [73]:
# Once the implementation is finished the BSM model price should also work
price_spread = basket.basket_price_mc(
    strike, spot, vol, weights, texp, cor_m, intr=intr, divr=divr, bsm=True)
price_spread

8.513040087377917

In [74]:
# You also test Kirk's approximation
kirk = pf.BsmSpreadKirk(vol, cor=cor, divr=divr, intr=intr)
price_kirk = kirk.price(strike, spot, texp)
print(price_kirk, price_spread)

8.513225229545505 8.513040087377917


# <a id='toc5_'></a>[[To Do] Complete the implementation of basket_price_norm_analytic](#toc0_)
# <a id='toc6_'></a>[Compare the MC stdev of BSM basket prices from with and without CV](#toc0_)

In [110]:
# The basket option example from above
spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 1, 1, 1])/4
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

In [121]:
### Make sure that the analytic normal price is correctly implemented

price_norm_analytic = basket.basket_price_norm_analytic(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr)
print(price_norm_analytic)

19.330395569726363


In [113]:
basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, intr=intr, divr=divr,bsm=True)

21.74354348351958

In [122]:
# Run below about 100 times and get the mean and stdev

### Returns 2 prices, without CV and with CV 
price_basket = basket.basket_price_mc_cv(strike, spot, vol, weights, texp, cor_m)
print(price_basket)

19.330395569726363
[22.1646178  21.95283839]


In [124]:
m_repeat = 100
value0 = np.zeros(m_repeat) # not use cv
value1 = np.zeros(m_repeat) # using cv
for i in range(0,m_repeat):
    price_basket1 = basket.basket_price_mc_cv(strike, spot, vol, weights, texp, cor_m)
    value0[i] = price_basket1[0]
    value1[i] = price_basket1[1] 

In [125]:
print("without cv [mean:{0},std:{1}]----with cv [mean:{2}, std:{3}]".format(np.mean(value0), np.std(value0),np.mean(value1),np.std(value1)))

without cv [mean:21.726894975868777,std:0.6223809933362467]----with cv [mean:21.745301495857216, std:0.36831507617734194]
